In [1]:
import os
import sys
parent_dir="/Volumes/work/phd/MoLFormer_N2024"
sys.path.append(parent_dir)
parent_dir="/Midgard/home/farzantn/mambaforge/envs/MolTran_CUDA11_cuda/lib/python3.8"
sys.path.append(parent_dir)

In [2]:
import scipy.io as sio
import numpy as np
import pandas as pd

path='/local_storage/datasets/farzaneh/Olfaction/fmri_test/'
base_dir= '../../../../T5 EVO/fmri/Fahime'

In [3]:
def process_condition_data(condition_data):
    """
    Process condition data by extracting and reshaping it into a 1D NumPy array.

    Parameters:
    condition_data (dict): Dictionary containing condition data with the key 'names'.

    Returns:
    np.ndarray: 1D array of processed condition values.
    """
    condition = np.moveaxis(condition_data['names'], -1, 0)
    condition = np.array([int(i[0][0]) for i in condition])
    return condition


In [4]:
def response_to_df(roi,s_number):
    """
    Process ROI response data into a DataFrame for a given subject.

    Parameters:
    roi (str): Region of interest (ROI) identifier.
    s_number (int): Subject number.

    Returns:
    DataFrame: Combined DataFrame containing ROI responses and associated metadata.
    """
    s_dir = f'/S{s_number}/'
    df_roi_all = pd.DataFrame()
    total_responses = 0

    for dir in os.listdir(base_dir + s_dir):
        # Skip hidden directories
        if dir.startswith('.'): 
            continue

        roi_dir = f"{base_dir}{s_dir}{dir}/fullmat_{roi}.mat"
        condition_file = f"{base_dir}{s_dir}{dir}/conditions_NEMO_0{s_number}.mat"

        try:
            roi_data = sio.loadmat(roi_dir)['odor_responses']
            print(roi_data.shape, roi, s_number)

            # Reorganize and preprocess ROI data
            roi_data = np.moveaxis(roi_data, -1, 0)
            total_responses += roi_data.shape[0]
            roi_data = np.where(np.isnan(roi_data), None, roi_data)

            # Load and process condition data
            condition_data = sio.loadmat(condition_file)
            condition_names = process_condition_data(condition_data)

            # Create DataFrame for ROI responses
            df = pd.DataFrame([
                [list(roi_data[i, j, :]) for j in range(roi_data.shape[1])] 
                for i in range(roi_data.shape[0])
            ])

            # Add metadata columns
            df['condition'] = condition_names
            df['subject'] = s_number
            df['roi'] = roi
            df['run'] = dir.split('_')[-1]
            df['session'] = dir.split('_')[-3]
            df['set'] = dir.split('_')[-5]

            # Append to the cumulative DataFrame
            df_roi_all = pd.concat([df_roi_all, df], axis=0)

        except (NotADirectoryError, KeyError, FileNotFoundError) as e:
            print(f"Error processing directory {dir}: {e}")

    print("Total responses processed:", total_responses, "Rows in DataFrame:", df_roi_all.shape[0], roi, s_number)
    return df_roi_all


In [5]:
for i in [1,2,3]:
    for roi in [  'OFC'
        # ,'APC', 'Amygdala', 'PPC'
                  ]:
        df_subject_roi = response_to_df(roi,i)
        df_subject_roi.rename(columns={'condition': 'CID'}, inplace=True)
        value_columns = df_subject_roi.loc[:, :'CID'].columns[:-1]
        
# Melt the dataframe
        fmri_ev_melted = df_subject_roi.melt(
        id_vars=['CID', 'subject', 'roi', 'run', 'session', 'set'],  # Columns to keep as is
        value_vars=value_columns,                                   # Columns to unpivot
        var_name='voxel',                                     # Name for the new column indicating original column names
        value_name='fmri'                                          # Name for the new column indicating values
        )

        #dataframe to csv
        df_subject_roi.to_csv(base_dir+f"/fmri_{i}_{roi}.csv", index=False)

(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 100) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11, 90) OFC 1
(6278, 11,